<a href="https://colab.research.google.com/github/jcdevaney/pyAMPACTtutorials/blob/main/03_pyAMPACT_alignment_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>pyAMPACT Performance Data Functions</h1>

In [1]:
!git clone https://github.com/jcdevaney/pyAMPACTtutorials.git
from IPython.utils import io
print('Importing libraries...')
with io.capture_output() as captured:
    !pip install --upgrade pandas
    !pip install -i https://test.pypi.org/simple/ --no-deps pyampact==0.4.6
    import pyampact
import pandas as pd
import numpy as np
import librosa

Cloning into 'pyAMPACTtutorials'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 263 (delta 58), reused 17 (delta 14), pack-reused 177
Receiving objects: 100% (263/263), 17.37 MiB | 20.40 MiB/s, done.
Resolving deltas: 100% (119/119), done.
Importing libraries...


Import and align symbolic and audio files (see 01-pyAMPACT_introduction.ipynb for details)

In [4]:
audio_file = '/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.wav'
midi_file = '/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.krn'

y, original_sr = librosa.load(audio_file)

piece = pyampact.Score(midi_file)

means = pd.read_csv('/content/pyAMPACTtutorials/test_files/means.csv', sep=' ').values
covars = pd.read_csv('/content/pyAMPACTtutorials/test_files/covars.csv', sep=' ').values

target_sr = 4000
win_ms = 100
hop_length = 32
width = 3
n_harm = 3

res, dtw, spec, nmat = pyampact.run_alignment(
    y, original_sr, piece, means, covars, piece.nmats(), width, target_sr, n_harm, win_ms, hop_length)


	Detected and imported these spine types:
		 function harm 



/usr/local/lib/python3.10/dist-packages/pyampact/alignmentUtils.py:452: ComplexWarning: Casting complex values to real discards the imaginary part
  S[r, :] = np.sqrt(M[:, r] @ D) / nDc


In [5]:
# Calculate an instananeous frequency-gram of the audio signal
freqs, times, mags, f0_values, mags_mat = pyampact.ifgram(audiofile=audio_file, tsr=target_sr, win_ms=win_ms)
mags_db = librosa.amplitude_to_db(mags, ref=np.max)

# Estimate F0 values
f0_values, sig_pwr = pyampact.calculate_f0_est(audio_file, hop_length, win_ms, target_sr)
sig_pwr = mags ** 2 # power of signal, magnitude/amplitude squared

# Prune NaN and zero values from estimated F0 and power
f0_values = f0_values[~np.isnan(f0_values)]
sig_pwr = sig_pwr[sig_pwr != 0]

# pyampact.data_compilation(f0_values, sig_pwr, mags_mat, nmat, target_sr, hop_length, audio_file, y)